# SH_Event_display

This notebook provides interactive 3D visualization of individual Solid Hydrogen events using ROOT data and Plotly.

## Overview
It loads reconstructed event data from a ROOT file and displays a user-selected event in 3D, showing:
- **Detector geometry**: ECAL cylindrical calorimeter (inner and outer surfaces)
- **Event topology**: 
  - Interaction vertex position
  - Muon track start and second hit positions
  - Muon true trajectory points through the detector
- **Calorimeter activity**:
  - ECAL cluster positions
  - Neutron segment start positions
- **Momentum vectors**:
  - True neutron momentum direction (black arrow)
  - Kinematically predicted neutron momentum (red arrow)

The visualization is interactive and can be rotated, zoomed, and panned. Optionally saves event displays as standalone HTML files.

## Usage
Set the input ROOT file path and output directory in the first code cell. Modify the `event_index` parameter to select different events, then run cells sequentially.

## Dependencies
- ROOT (with PyROOT support)
- Plotly (for interactive 3D visualization)
- NumPy
- Local helper module: `df_functions_utils.py`


In [ ]:
import ROOT
from ROOT import TCanvas
from ROOT import TLegend
from ROOT import RDataFrame
from ROOT import kRed, kBlue
from ROOT import gStyle
import numpy as np
import plotly.graph_objects as go
import df_functions_utils as df_utils

df = RDataFrame("Rough_tree", "/storage/gpfs_data/neutrino/users/battisti/hydrogen_analysis_tests/Solid_Hydrogen2/Snap_file_processing/Processed_tree.root")
Out_dir = "Plots/SH_Display/"

In [ ]:
### Uncoment for fast testing
df = df.Range(1000)

# Generating df with additional columns

### List of dataframes:
- df = contains Rough_tree with all its columns
- df_dipslay =  df + additional columns needed for display + flags that identify the filters

### Filters flags for df_display:
- inside the STT volume (volume_filter)
- successufull reconstruction (reco_filter)
- one track (track_filter)
- neutron space correspondence (space_filter, done with clusters) (spaceTRUE_filter, done with hit segment starts)
- neutron time correspondence (time_filter, done with clusters) (timeTRUE_filter, done with hit segment starts)
- events CC-QE with antimuon and neutron in the final state, on H (true_filter) 

In [ ]:
df_display = df_utils.df_add_columns_for_display(df)


# Plotting

In [ ]:
###Comment to avoid interactivity
%jsroot

In [ ]:
# --- Parametri cilindri (in mm) ---
r_out, r_in = 2230.0, 2000.0
x_min, x_max = -2150.0, 2150.0

# Centro del cilindro
x_c, y_c, z_c = 0.0, -2384.73, 23910.0

#event_index = 10


event_index = 227 #good

evt_df = df_display.Range(event_index,event_index+1)

vtx_pos = evt_df.AsNumpy(["vertex_x","vertex_y","vertex_z"])
p1 = evt_df.AsNumpy(["x1","y1","z1"])
p2 = evt_df.AsNumpy(["x2","y2","z2"])
traj_point = evt_df.AsNumpy(["true_x_pos", "true_y_pos", "true_z_pos"])
ecal_point = evt_df.AsNumpy(["x_ecal", "y_ecal", "z_ecal"])
seg_point = evt_df.AsNumpy(["true_n_startX", "true_n_startY", "true_n_startZ"])
true_mom = evt_df.AsNumpy(["px_end","py_end","pz_end"])
pred_mom = evt_df.AsNumpy(["px_end2","py_end2","pz_end2"])
# segment = evt_df


traj = {k: list(traj_point[k][0]) for k in traj_point}
ecal = {k: list(ecal_point[k][0]) for k in ecal_point}
seg = {k: list(seg_point[k][0]) for k in seg_point}

# true_n_startX"

# --- Griglia cilindri ---
theta = np.linspace(0, 2*np.pi, 100)
x = np.linspace(x_min, x_max, 50)
Theta, X = np.meshgrid(theta, x)

# --- Coordinate cilindri ---
Y_out = y_c + r_out * np.cos(Theta)
Z_out = z_c + r_out * np.sin(Theta)

Y_in  = y_c + r_in  * np.cos(Theta)
Z_in  = z_c + r_in  * np.sin(Theta)

# --- Creazione figura Plotly ---
fig = go.Figure()

# Cilindro esterno
fig.add_surface(
    x=X + x_c, y=Y_out, z=Z_out,
    colorscale='Blues', opacity=0.5, showscale=False,
    name='Outer cylinder'
)

# Cilindro interno
fig.add_surface(
    x=X + x_c, y=Y_in, z=Z_in,
    colorscale='Blues', opacity=0.5, showscale=False,
    name='Inner cylinder'
)

#vertex
fig.add_trace(go.Scatter3d(
    x= vtx_pos["vertex_x"], y=vtx_pos["vertex_y"], z=vtx_pos["vertex_z"],
    mode='markers+text',
    #text=labels,
    textposition='top center',
    marker=dict(size=6, color='orange', symbol='circle'),
    name='Vtx. position'
))

#firt hit
fig.add_trace(go.Scatter3d(
    x= p1["x1"], y=p1["y1"], z=p1["z1"],
    mode='markers+text',
    #text=labels,
    textposition='top center',
    marker=dict(size=6, color='green', symbol='circle'),
    name='p1 position'
))

#second hit
fig.add_trace(go.Scatter3d(
    x= p2["x2"], y=p2["y2"], z=p2["z2"],
    mode='markers+text',
    #text=labels,
    textposition='top center',
    marker=dict(size=6, color='green', symbol='circle'),
    name='p2 position'
))

#trajectory points
fig.add_trace(go.Scatter3d(
    x= traj["true_x_pos"], y=traj["true_y_pos"], z=traj["true_z_pos"],
    mode='markers',
    marker=dict(size=6, color='orange', symbol='circle'),
    name='Trajectory points'
))

#cluster position
fig.add_trace(go.Scatter3d(
    x= ecal["x_ecal"], y=ecal["y_ecal"], z=ecal["z_ecal"],
    mode='markers',
    marker=dict(size=6, color='blue', symbol='circle'),
    name='Ecal points'
))

#seg position
fig.add_trace(go.Scatter3d(
    x= seg["true_n_startX"], y=seg["true_n_startY"], z=seg["true_n_startZ"],
    mode='markers',
    marker=dict(size=6, color='pink', symbol='circle'),
    name='segment points'
))

x0 = vtx_pos["vertex_x"][0]
y0 = vtx_pos["vertex_y"][0]
z0 = vtx_pos["vertex_z"][0]

x1 = true_mom["px_end"][0]
y1 = true_mom["py_end"][0]
z1 = true_mom["pz_end"][0]
print(f"({x1}, {y1}, {z1})")
f = 5000

x_new = x0 + f * (x1)
y_new = y0 + f * (y1)
z_new = z0 + f * (z1)

fig.add_trace(go.Scatter3d(

    x=[x0, x_new],
    y=[y0, y_new],
    z=[z0, z_new],

    mode='lines',
    line=dict(color='black', width=8),
    name='true neutron momentum'
))


x1_p = pred_mom["px_end2"][0]
y1_p = pred_mom["py_end2"][0]
z1_p = pred_mom["pz_end2"][0]

x_new2 = x0 + f * (x1_p)
y_new2 = y0 + f * (y1_p)
z_new2 = z0 + f * (z1_p)

#pred momentum
fig.add_trace(go.Scatter3d(
    x=[x0, x_new2],
    y=[y0, y_new2],
    z=[z0, z_new2],

    mode='lines',
    line=dict(color='red', width=8),
    name='pred neutron momentum'
))



fig.update_layout(
    title='ECAL Cylinders with 3 Variable 3D Points',
    scene=dict(
        xaxis_title='X [mm]',
        yaxis_title='Y [mm]',
        zaxis_title='Z [mm]',
        aspectmode='data'
    ),
    width=900,
    height=700
)

fig.show()

In [ ]:
### Uncomment to save the figure
fig.write_html(Out_dir + f"event_{event_index}_display.html")